## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`

In [2]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [ ]:
mlflow.list_experiments()

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-2")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/27 05:53:40 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-2' does not exist. Creating a new experiment.


default artifacts URI: 'file:///C:/Users/ArunGanesan/mlops-zoomcamp-garun/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/a68ae5197ecd451dbf31dd34f0f0e778/artifacts'


In [ ]:
mlflow.list_experiments()

### Interacting with the model registry

In [5]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [6]:
client.list_registered_models()

AttributeError: 'MlflowClient' object has no attribute 'list_registered_models'

In [7]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

AttributeError: 'MlflowClient' object has no attribute 'list_run_infos'

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.accuracy >= 0.9',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.accuracy ASC"]
)

In [9]:
runs

[<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "a68ae5197ecd451dbf31dd34f0f0e778", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2024-05-27 00:23:41.235101", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.12.2", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.5.0", '
                              '"serialization_format": "cloudpickle", "code": '
                              'null}}, "model_uuid": '
                              

In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, accuracy: {run.data.metrics['accuracy']:.4f}" )
    

run id: a68ae5197ecd451dbf31dd34f0f0e778, accuracy: 0.9600


In [12]:
run_id = 'a68ae5197ecd451dbf31dd34f0f0e778'
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/27 06:02:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716769973377, current_stage='None', description='', last_updated_timestamp=1716769973377, name='iris-classifier', run_id='a68ae5197ecd451dbf31dd34f0f0e778', run_link='', source='file:///C:/Users/ArunGanesan/mlops-zoomcamp-garun/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/a68ae5197ecd451dbf31dd34f0f0e778/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>